<a href="https://colab.research.google.com/github/hr7git/Data-Analysis-with-Open-Source/blob/main/15%EA%B0%95_%EB%B3%B4%EC%B6%A9%EB%AC%B8%EC%A0%9C_%ED%95%B4%EC%84%A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 **'3단계 빌드업 숙제'**의 해설 코드입니다.

각 단계에서 **왜 이런 코드를 쓰는지**에 대한 이유를 주석으로 상세히 달았습니다.

---

### ✅ 숙제 1 해설: 데이터 수집과 이동평균선(MA)

이동평균선은 주가의 '평균'을 구함으로써 들쑥날쑥한 차트를 부드럽게 만들어 추세를 보게 해줍니다.

```python
import yfinance as yf
import pandas as pd

# 삼성전자 데이터 수집
df = yf.download('005930.KS', start='2023-01-01')

# [해설] rolling 함수는 '윈도우(범위)'를 이동시키며 연산합니다.
# 1. 5일 이동평균선: 최근 5일치 종가의 평균
df['MA5'] = df['Close'].rolling(window=5).mean()

# 2. 20일 이동평균선: 최근 20일치 종가의 평균
df['MA20'] = df['Close'].rolling(window=20).mean()

# 결과 확인 (데이터가 없는 초기 날짜는 NaN으로 표시됩니다)
print(df[['Close', 'MA5', 'MA20']].tail())

```

---

### ✅ 숙제 2 해설: 골든크로스 전략 로직

수학적 조건문을 데이터프레임 전체에 적용하는 방법입니다. `np.where`는 엑셀의 `IF` 함수와 똑같은 역할을 합니다.

```python
import numpy as np

# [해설] np.where(조건, 참일때값, 거짓일때값)
# 5일 선이 20일 선을 돌파하면 매수 신호(1)를 보냅니다.
df['Signal'] = np.where(df['MA5'] > df['MA20'], 1, 0)

# [보충] 신호가 0에서 1로 변하는 지점이 실제 '매수 타이밍'입니다.
print("최근 매수/대기 신호 현황:")
print(df['Signal'].tail(10))

```

---

### ✅ 숙제 3 해설: AI(랜덤포레스트) 예측 모델

과거의 특징(Features)을 정답(Target)과 연결하는 '학습' 과정의 해설입니다.

```python
from sklearn.ensemble import RandomForestRegressor

# 1. 데이터 정제: 이동평균선 계산으로 생긴 빈칸(NaN)과
# 마지막 날의 타겟(내일 가격이 없으므로)을 제거합니다.
df['Target'] = df['Close'].shift(-1)
df_clean = df.dropna()

# 2. 독립변수(X)와 종속변수(y) 설정
# AI가 공부할 재료: MA5, MA20 (과거 패턴)
X = df_clean[['MA5', 'MA20']]
# AI가 맞출 정답: Target (내일의 실제 가격)
y = df_clean['Target']

# 3. 모델 생성 및 학습
# n_estimators는 나무(결정 트리)의 개수입니다. 많을수록 정교해집니다.
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X, y)

# 4. 가장 최근 데이터를 넣어 내일 가격 예측
last_features = X.tail(1)
prediction = model.predict(last_features)

print(f"--- AI 예측 결과 ---")
print(f"오늘 종가: {df['Close'].iloc[-1]:.0f}원")
print(f"AI가 예상한 내일 종가: {prediction[0]:.0f}원")

```

---

### 💡  핵심 포인트 (Tip)

1. **숙제 1 (데이터 전처리):** "맨 앞쪽 데이터에 `NaN`이 나오는 이유는 5일치 데이터가 쌓이기 전까지는 평균을 구할 수 없기 때문이야."
2. **숙제 2 (알고리즘):** "우리가 만든 이 규칙이 바로 '트레이딩 봇'의 가장 기초적인 뇌가 되는 거야."
3. **숙제 3 (머신러닝):** "AI는 주식의 미래를 예언하는 게 아니라, 과거에 5일선과 20일선이 이 정도 값이었을 때 다음 날 주가가 어떻게 변했었는지를 **통계적으로 기억**하는 거야."

자신의 코드와 비교하며 **데이터 분석의 전체 워크플로우**를 완벽히 이해할 수 있을 것입니다.